In [4]:
import pandas as pd
import numpy as np
import re
import json

In [7]:
V1 = pd.read_excel('data/data_A.xlsx')
V2 = pd.read_excel('data/data_B.xlsx')
V3 = pd.read_excel('data/data_C.xlsx')
V4 = pd.read_excel('data/data_D.xlsx')
V5 = pd.read_excel('data/data_E.xlsx')
V6 = pd.read_excel('data/data_F.xlsx')

In [11]:
V = pd.concat([V1,V2,V3,V4,V5,V6])
V = V[['제목','영상길이','링크','조회수']]

In [12]:
V['동영상_ID'] = V['링크'].str[-11:]

In [4]:
#유투브 웹사이트의 html 데이터를 추출해와서 그중에서 제목,조회수,업로드 일 등의 정보를 추출할 수 있도록 준비

from requests_html import HTMLSession 
from bs4 import BeautifulSoup as bs # importing BeautifulSoup

#nest_asyncio.apply() ASNC문제가 HTMLSession이용중에 발생하여 이를 조치하기 위한 모듈
import asyncio
if asyncio.get_event_loop().is_running():
    import nest_asyncio
    nest_asyncio.apply()
#url = 웹사이트 주소
url = 'https://www.youtube.com/watch?v=oUvPWwmvoPM'

session = HTMLSession()
r = session.get(url) 
#입력한 url에서 html정보를 가져옴

#soup = 추출한 html 정보
html_str = r.text
soup = bs(r.html.html, "html.parser")
#soup.find_all("meta") 를 통해 html정보에서 메타데이터(조회수,제목 등)만을 추출
#soup.find("meta", itemprop="datePublished")["content"] 의 경우 업로드일(영상출시일)을 추출함

In [77]:
#데이터베이스에 '업로드일'열을 추가 후, for문을 통해 각 동영상ID를 통한 url입력을 통해 페이지에서 업로드일을 추출

V['업로드일'] = np.nan
V['실제조회수'] = np.nan
V['설명'] = np.nan
V['좋아요'] = np.nan

for i in range(len(V)):
    url = 'https://www.youtube.com/watch?v=' + V.동영상_ID[i]
    r = session.get(url)
    soup = bs(r.html.html, "html.parser")
    V['업로드일'][i] = soup.find("meta", itemprop="datePublished")["content"]
    V['실제조회수'][i] = soup.find("meta", itemprop="interactionCount")["content"]
    V['설명'][i] = soup.find("meta", itemprop="description")["content"]
    try:
        data = re.search(r"var ytInitialData = ({.*?});", soup.prettify()).group(1)
        data_json = json.loads(data)
        videoPrimaryInfoRenderer = data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents'][0]['videoPrimaryInfoRenderer']
        videoSecondaryInfoRenderer = data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents'][1]['videoSecondaryInfoRenderer']
        # number of likes
        likes_label = videoPrimaryInfoRenderer['videoActions']['menuRenderer']['topLevelButtons'][0]['toggleButtonRenderer']['defaultText']['accessibility']['accessibilityData']['label'] # "No likes" or "###,### likes"
        likes_str = likes_label.split(' ')[1].replace(',','')
        V['좋아요'][i] = '0' if likes_str == 'No' else likes_str
    except:
        V['좋아요'][i] = 'ERROR'
        continue

C:\Users\jpark\AppData\Local\Temp\ipykernel_20252\2030871249.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  V['좋아요'][i] = '0' if likes_str == 'No' else likes_str


In [78]:
V.to_excel('data/브이로그디테일추출.xlsx', index = False)